In [114]:
import pandas as pd
import tweepy
import numpy as np
from textblob import TextBlob

from datetime import datetime
from datetime import timedelta
from dataclasses import make_dataclass

import re
import emoji
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import f1_score, accuracy_score

import pandas_datareader as web
import matplotlib.pyplot as plt
import mplfinance as mpf
import seaborn as sns
import bitfinex as btf
import time

import nltk
from nltk.stem.porter import PorterStemmer
from wordcloud import WordCloud
from nltk.corpus import stopwords
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score

compT, compR, compG = [], [],[]
negT, negR, negG = [], [],[]
posT, posR, posG = [], [],[]
neuT, neuR, neuG = [], [],[]
SIAT,SIAR, SIAG = 0, 0, 0

from sklearn.naive_bayes import GaussianNB

import pandas as pd
import pandas_datareader as web
import tweepy
import numpy as np
from textblob import TextBlob

from datetime import datetime

from datetime import timedelta
from dataclasses import make_dataclass

import re
import emoji
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer

from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis

import matplotlib.pyplot as plt
import mplfinance as mpf
import seaborn as sns
import bitfinex as btf
import time

import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer 
from nltk import ngrams
from collections import Counter

import spacy 
from spacy import displacy

from GoogleNews import GoogleNews
from newspaper import Article
from newspaper import Config

import praw


In [128]:
#Crear una funcion para guardar puntajes de sentimiento con vaderSentiment
def getSIA(text):
  sia = SentimentIntensityAnalyzer()
  sentiment = sia.polarity_scores(text)
  return sentiment
def setSIA(field,pos,neg,neu,comp):
    for i in range(0, len(field)):
      SIA = getSIA(field[i])
      comp.append(SIA['compound'])
      neg.append(SIA['neg'])
      neu.append(SIA['neu'])
      pos.append(SIA['pos'])
    return [pos,neg,neu,comp]

In [129]:
def labeleador(df):
    labels=[]
    for i in range(len(df)):
        if df['Open'][i]:
            if df['Open'][i]!="" and df['Close'][i]!="":
                if int(float(df['Open'][i]))<int(float(df['Close'][i])):
                    labels.append(1)
                else:
                    labels.append(0)
    df['Label']=labels
    return df

In [130]:
def limpiaVacios(df):
    for i in range(len(df)):
        if "nameUser" in df['nameUser'][i]:
            df = df.drop(index=i)
    df = df.drop_duplicates()
    df = df.reset_index(drop=True)
    return df
def limpiaErroresR(df):
    for i in range(len(df)):
        if "Title" in df['Title'][i]:
            df = df.drop(index=i)
    df = df.drop_duplicates()
    df = df.reset_index(drop=True)
    return df
def limpiaErroresG(df):
    for i in range(len(df)):
        if "Title" in df['Title'][i]:
            df = df.drop(index=i)
    df = df.drop_duplicates()
    df = df.reset_index(drop=True)
    return df

In [131]:
df_tweet = pd.read_csv('exportTweetData.csv')
df_market = pd.read_csv('exportMarketData.csv')
df_reddit = pd.read_csv('exportRedditData.csv')
df_google = pd.read_csv('exportNewsData.csv')


In [132]:
dataSetCombinadoTwitter = df_tweet.merge(df_market, how='inner', on='Date')
dataSetCombinadoReddit = df_reddit.merge(df_market,how='inner', on='Date')
dataSetCombinadoGoogle = df_google.merge(df_market, how='inner',on='Date')

In [133]:
dataSetCombinadoTwitter = labeleador(dataSetCombinadoTwitter)

dataSetCombinadoReddit = limpiaVacios(dataSetCombinadoReddit)
dataSetCombinadoReddit = labeleador(dataSetCombinadoReddit)
dataSetCombinadoReddit = limpiaErroresR(dataSetCombinadoReddit)

dataSetCombinadoGoogle = limpiaErroresG(dataSetCombinadoGoogle)
#dataSetCombinadoGoogle
dataSetCombinadoGoogle = labeleador(dataSetCombinadoGoogle)


In [134]:
resultadosSIAT = setSIA(dataSetCombinadoTwitter['Tweets'],posT,negT,neuT,compT)
resultadosSIAR = setSIA(dataSetCombinadoReddit['Title'],posR,negR,neuR,compR)
resultadosSIAG = setSIA(dataSetCombinadoGoogle['Desc'],posG,negG,neuG,compG)

compT = resultadosSIAT[0]
negT = resultadosSIAT[1]
neuT = resultadosSIAT[2]
posT = resultadosSIAT[3]

compR = resultadosSIAR[0]
negR = resultadosSIAR[1]
neuR = resultadosSIAR[2]
posR = resultadosSIAR[3]

compG = resultadosSIAG[0]
negG = resultadosSIAG[1]
neuG = resultadosSIAG[2]
posG = resultadosSIAG[3]

dataSetCombinadoTwitter['Compound'] = compT
dataSetCombinadoTwitter['Negative'] = negT
dataSetCombinadoTwitter['Neutral'] = neuT
dataSetCombinadoTwitter['Positive'] = posT

dataSetCombinadoReddit['Compound'] = compR
dataSetCombinadoReddit['Negative'] = negR
dataSetCombinadoReddit['Neutral'] = neuR
dataSetCombinadoReddit['Positive'] = posR

dataSetCombinadoGoogle['Compound'] = compG
dataSetCombinadoGoogle['Negative'] = negG
dataSetCombinadoGoogle['Neutral'] = neuG
dataSetCombinadoGoogle['Positive'] = posG

In [135]:
keep_columns = ['Open','High','Low','Volume','Compound','Negative','Neutral','Positive','Label']
trainDFLinealTwitter = dataSetCombinadoTwitter[keep_columns]
trainDFLinealReddit = dataSetCombinadoReddit[keep_columns]
trainDFLinealGoogle = dataSetCombinadoGoogle[keep_columns]

In [164]:
#Para entrenamiento de programa con modelo lineal
#Para modelo Lineal necesitamos np.array
X_twitter = trainDFLinealTwitter
X_twitter = np.array(X_twitter.drop('Label',1))

#Create the target data set
Y_twitter = np.array(dataSetCombinadoTwitter['Label'])

In [165]:
#Para entrenamiento de programa con modelo lineal
#Para modelo Lineal necesitamos np.array
X_reddit = trainDFLinealReddit
X_reddit = np.array(X_reddit.drop('Label',1))

#Create the target data set
Y_reddit = np.array(dataSetCombinadoReddit['Label'])

In [166]:
#Para entrenamiento de programa con modelo lineal
#Para modelo Lineal necesitamos np.array
X_google = trainDFLinealGoogle
X_google = np.array(X_google.drop('Label',1))

#Create the target data set
Y_google = np.array(dataSetCombinadoGoogle['Label'])

In [167]:
#Set de entrenamiento Twitter
x_train_t, x_test_t, y_train_t, y_test_t = train_test_split(X_twitter,Y_twitter, test_size = 0.2, random_state = 0)

In [168]:
modelLDT = LinearDiscriminantAnalysis().fit(x_train_t,y_train_t)
predLinearT = modelLDT.predict(x_test_t)
print(classification_report(y_test_t, predLinearT))
print(accuracy_score(y_test_t,predLinearT))

              precision    recall  f1-score   support

           0       0.81      0.85      0.83        41
           1       0.89      0.86      0.88        59

    accuracy                           0.86       100
   macro avg       0.85      0.86      0.86       100
weighted avg       0.86      0.86      0.86       100

0.86


In [147]:
#Set de entrenamiento Reddit
#x_train_r, x_test_r, y_train_r, y_test_r = train_test_split(X_reddit,Y_reddit, test_size = 0.2, random_state = 0)

In [148]:
#model = LinearDiscriminantAnalysis().fit(x_train_r,y_train_r)
#predLinearR = model.predict(x_test_r)
#print(classification_report(y_test_r, predLinearR))

In [149]:
#Set de entrenamiento Google
x_train_g, x_test_g, y_train_g, y_test_g = train_test_split(X_google,Y_google, test_size = 0.2, random_state = 0)

In [151]:
model = LinearDiscriminantAnalysis().fit(x_train_g,y_train_g)
predLinearG = model.predict(x_test_g)
print(classification_report(y_test_g, predLinearG))
print(accuracy_score(y_test_g,predLinearG))

              precision    recall  f1-score   support

           0       1.00      0.50      0.67         2
           1       0.80      1.00      0.89         4

    accuracy                           0.83         6
   macro avg       0.90      0.75      0.78         6
weighted avg       0.87      0.83      0.81         6

0.8333333333333334


In [152]:
#Bagg of word para twitter
bow_vectorizerT = CountVectorizer(max_df=0.9, min_df=2, max_features=1000, stop_words='english')
bowT = bow_vectorizerT.fit_transform(dataSetCombinadoTwitter['Tweets'])


In [153]:
#Bagg of word para google
bow_vectorizerG = CountVectorizer(max_df=0.9, min_df=2, max_features=1000, stop_words='english')
bowG = bow_vectorizerG.fit_transform(dataSetCombinadoGoogle['Desc'])

In [154]:
#Bagg of word para reddit
#bow_vectorizerR = CountVectorizer(max_df=0.9, min_df=2, max_features=1000, stop_words='english')
#bowR = bow_vectorizerR.fit_transform(dataSetCombinadoReddit['Title'])

In [155]:
#Para entrenamiento de programa con modelo regresivo Twitter
x_train_t, x_test_t, y_train_t, y_test_t = train_test_split(bowT,dataSetCombinadoTwitter['Label'], random_state = 42, test_size=0.25)

In [156]:
#Para entrenamiento de programa con modelo regresivo Google
x_train_g, x_test_g, y_train_g, y_test_g = train_test_split(bowG,dataSetCombinadoGoogle['Label'], random_state = 42, test_size=0.25)

In [157]:
#Para entrenamiento de programa con modelo regresivo Reddit
#x_train, x_test, y_train, y_test = train_test_split(bowR,dataSetCombinado['Label'], random_state = 42, test_size=0.25)

In [158]:
modelLRT = LogisticRegression().fit(x_train_t,y_train_t)
predRegT = modelLRT.predict(x_test_t)
print(classification_report(y_test_t, predRegT))
print(accuracy_score(y_test_t,predRegT))

              precision    recall  f1-score   support

           0       0.65      0.66      0.65        53
           1       0.75      0.74      0.74        72

    accuracy                           0.70       125
   macro avg       0.70      0.70      0.70       125
weighted avg       0.70      0.70      0.70       125

0.704


In [159]:
modelLRG = LogisticRegression().fit(x_train_g,y_train_g)
predRegG = modelLRG.predict(x_test_g)
print(classification_report(y_test_g, predRegG))
print(accuracy_score(y_test_g,predRegG))

              precision    recall  f1-score   support

           0       0.00      0.00      0.00         1
           1       0.86      1.00      0.92         6

    accuracy                           0.86         7
   macro avg       0.43      0.50      0.46         7
weighted avg       0.73      0.86      0.79         7

0.8571428571428571


C:\Users\christian\miniconda3\lib\site-packages\sklearn\metrics\_classification.py:1248: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\christian\miniconda3\lib\site-packages\sklearn\metrics\_classification.py:1248: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\christian\miniconda3\lib\site-packages\sklearn\metrics\_classification.py:1248: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [26]:
#newPred = model.predict(df_tweet['Tweets'])

In [87]:
dataT = dataSetCombinadoTwitter['Tweets'].tolist()
lblT = dataSetCombinadoTwitter['Label'].tolist()

dataR = dataSetCombinadoReddit['Title'].tolist()
lblR = dataSetCombinadoReddit['Label'].tolist()

dataG = dataSetCombinadoGoogle['Desc'].tolist()
lblG = dataSetCombinadoGoogle['Label'].tolist()

In [88]:
#RandomForest
vectorizerT = TfidfVectorizer(max_features = 2500, max_df=0.8, stop_words = stopwords.words('english'))
dataT = vectorizerT.fit_transform(dataT).toarray()

#vectorizerR = TfidfVectorizer(max_features = 2500, max_df=0.8, stop_words = stopwords.words('english'))
#dataR = vectorizerR.fit_transform(dataR).toarray()

vectorizerG = TfidfVectorizer(max_features = 2500, max_df=0.8, stop_words = stopwords.words('english'))
dataG = vectorizerG.fit_transform(dataG).toarray()

In [89]:
x_train_t, x_test_t, y_train_t, y_test_t = train_test_split(dataT, lblT, test_size = 0.2, random_state = 0)

#x_train_r, x_test_r, y_train_r, y_test_r = train_test_split(dataR, lblR, test_size = 0.2, random_state = 0)

x_train_g, x_test_g, y_train_g, y_test_g = train_test_split(dataG, lblG, test_size = 0.2, random_state = 0)

In [90]:
text_classifierT = RandomForestClassifier(n_estimators = 200, random_state = 0)
text_classifierT.fit(x_train_t, y_train_t)

#text_classifierR = RandomForestClassifier(n_estimators = 200, random_state = 0)
#text_classifierR.fit(x_train_r, y_train_r)

text_classifierG = RandomForestClassifier(n_estimators = 200, random_state = 0)
text_classifierG.fit(x_train_g, y_train_g)

RandomForestClassifier(n_estimators=200, random_state=0)

In [91]:
predictionsT = text_classifierT.predict(x_test_t)

#predictionsR = text_classifierR.predict(x_test_r)

predictionsG = text_classifierG.predict(x_test_g)

In [127]:
print(confusion_matrix(y_test_t,predictionsT))
print(classification_report(y_test_t,predictionsT))
print(accuracy_score(y_test_t,predictionsT))

ValueError: Found input variables with inconsistent numbers of samples: [125, 100]

In [93]:
#print(confusion_matrix(y_test_r,predictionsR))
#print(classification_report(y_test_r,predictionsR))
#print(accuracy_score(y_test_r,predictionsR))

In [94]:
print(confusion_matrix(y_test_g,predictionsG))
print(classification_report(y_test_g,predictionsG))
print(accuracy_score(y_test_g,predictionsG))

[[0 2]
 [0 4]]
              precision    recall  f1-score   support

           0       0.00      0.00      0.00         2
           1       0.67      1.00      0.80         4

    accuracy                           0.67         6
   macro avg       0.33      0.50      0.40         6
weighted avg       0.44      0.67      0.53         6

0.6666666666666666


C:\Users\christian\miniconda3\lib\site-packages\sklearn\metrics\_classification.py:1248: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\christian\miniconda3\lib\site-packages\sklearn\metrics\_classification.py:1248: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\christian\miniconda3\lib\site-packages\sklearn\metrics\_classification.py:1248: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [124]:
#Gauseano
modelT = GaussianNB()
modelG = GaussianNB()
modelR = GaussianNB()

In [125]:
modelT.fit(x_train_t, y_train_t)
modelG.fit(x_train_g, y_train_g)

TypeError: A sparse matrix was passed, but dense data is required. Use X.toarray() to convert to a dense numpy array.

In [126]:
predictionT = modelT.predict(x_test_t)
print(x_test_t)
predictionG = modelG.predict(x_test_g)

NotFittedError: This GaussianNB instance is not fitted yet. Call 'fit' with appropriate arguments before using this estimator.

In [98]:
print(confusion_matrix(y_test_t,predictionsT))
print(classification_report(y_test_t,predictionsT))
print(accuracy_score(y_test_t,predictionsT))

[[24 17]
 [37 22]]
              precision    recall  f1-score   support

           0       0.39      0.59      0.47        41
           1       0.56      0.37      0.45        59

    accuracy                           0.46       100
   macro avg       0.48      0.48      0.46       100
weighted avg       0.49      0.46      0.46       100

0.46


In [99]:
print(confusion_matrix(y_test_g,predictionsG))
print(classification_report(y_test_g,predictionsG))
print(accuracy_score(y_test_g,predictionsG))

[[0 2]
 [0 4]]
              precision    recall  f1-score   support

           0       0.00      0.00      0.00         2
           1       0.67      1.00      0.80         4

    accuracy                           0.67         6
   macro avg       0.33      0.50      0.40         6
weighted avg       0.44      0.67      0.53         6

0.6666666666666666


C:\Users\christian\miniconda3\lib\site-packages\sklearn\metrics\_classification.py:1248: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\christian\miniconda3\lib\site-packages\sklearn\metrics\_classification.py:1248: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\christian\miniconda3\lib\site-packages\sklearn\metrics\_classification.py:1248: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [ ]:
#Mejor Score
#Twitter -> Regression ~= 70
#Google -> Regression ~= 85

In [117]:
#Acceso a Twitter API
consumerKey = "TwlSxQ7X0qOJ9XPYi0RRGFmPy"
consumerSecret = "4QCBGcgWOOhHWXe0SwDu5UMVGVSZvnZujkHhacliUQeBigM7ZS"
accessToken = "1083093878893432836-HW9vv7t1xihZvZvq0YdomkNKPzFnWe"
accessTokenSecret = "PW8gxDGBCaNcprbkIb0uEQstg5tUzL55KRWEOI8Vqz2U9"

authenticate = tweepy.OAuthHandler(consumerKey, consumerSecret)
authenticate.set_access_token(accessToken, accessTokenSecret)
api = tweepy.API(authenticate, wait_on_rate_limit=True)


In [119]:
#LimpiaTexto
def cleanText(twt):
    EMOJI_PATTERN = re.compile(
    "["
    "\U0001F1E0-\U0001F1FF"  # flags (iOS)
    "\U0001F300-\U0001F5FF"  # symbols & pictographs
    "\U0001F600-\U0001F64F"  # emoticons
    "\U0001F680-\U0001F6FF"  # transport & map symbols
    "\U0001F700-\U0001F77F"  # alchemical symbols
    "\U0001F780-\U0001F7FF"  # Geometric Shapes Extended
    "\U0001F800-\U0001F8FF"  # Supplemental Arrows-C
    "\U0001F900-\U0001F9FF"  # Supplemental Symbols and Pictographs
    "\U0001FA00-\U0001FA6F"  # Chess Symbols
    "\U0001FA70-\U0001FAFF"  # Symbols and Pictographs Extended-A
    "\U00002702-\U000027B0"  # Dingbats
    "\U000024C2-\U0001F251" 
    "]+"
    )
    twt = re.sub('#[A-Za-z0-9]+','',twt) #Removes any string with hash
    twt = re.sub(r'[0-9]+', '', twt)
    twt = re.sub('@[A-Za-z0-9]+','',twt) #Removes any string with at
    twt = re.sub('\\n','',twt) #Remove \n caracter
    twt = EMOJI_PATTERN.sub(r'', twt)
    twt = twt.replace('!','').replace('-','').replace('_','')
    twt = re.sub('https?:\/\/\S+','',twt) #Removes hastag
    return twt
def cleanTime(time):
    return time.strftime("%Y-%m-%d %H:%M")
def cleanTimeStamp(time):
    return datetime.fromtimestamp(time).strftime('%Y-%m-%d %H:%M')

In [122]:
analyzedCrypto = 'BTC'
#CREAR DATASET DE TWEETS
cantidadDeTweetsBuscados = 500
#loopeo para obtener y almacenar info de todas las monedas

search_term = analyzedCrypto+' -filter:retweets'
#mySince = datetime.now()-timedelta(minutes=30)
#mySince = mySince.strftime("%Y-%m-%d %H:%M:%S")
#myUntil = datetime.now().strftime("%Y-%m-%d %H:%M:%S")
mySince="2021-07-2"
myUntil="2021-07-2"
tweets = tweepy.Cursor(api.search, q=search_term, lang ='en', since=mySince, until=myUntil, tweet_mode='extended').items(cantidadDeTweetsBuscados)
all_tweets = [tweet.full_text for tweet in tweets]

tweets = tweepy.Cursor(api.search, q=search_term, lang ='en', since=mySince, until=myUntil,  tweet_mode='extended').items(cantidadDeTweetsBuscados)
all_dates = [tweet.created_at for tweet in tweets]


formatoDataSetTweets = {'Date': all_dates, 'Tweets': all_tweets}
df_tweets = pd.DataFrame(data=formatoDataSetTweets)

df_tweets['Tweets'] = df_tweets['Tweets'].apply(cleanText)
df_tweets['Date'] = df_tweets['Date'].apply(cleanTime)

#Genero dataSet con valores financieros
api_v2 = btf.bitfinex_v2.api_v2()

#CREAR DATASET DE VALORES DE MERCADO
pair = analyzedCrypto
TIMEFRAME = '1m'

t_start = datetime(2020,7,2,0,0)
t_start = time.mktime(t_start.timetuple())*1000

t_stop = datetime(2021,7,2,0,0)
t_stop = time.mktime(t_stop.timetuple())*1000

result = api_v2.candles(symbol = 'BTCUSD', interval = TIMEFRAME, limit = 5000, start = t_start, end = t_stop)

camposDataFrame = ['Date','Open', 'Close', 'High', 'Low', 'Volume']
df_market = pd.DataFrame(result, columns=camposDataFrame)
df_market['Date'] = pd.to_datetime(df_market['Date'], unit='ms')
df_market['Date'] = df_market['Date'].apply(cleanTime)


googlenews=GoogleNews(start='07/2/2021',end='07/2/2021')
googlenews.search('Bitcoin')
result=googlenews.result()
camposDataFrame = ['title','media', 'datetime', 'desc']
df_news=pd.DataFrame(result,columns=camposDataFrame)
df_news['title']=df_news['title'].apply(cleanText)
df_news['desc']=df_news['desc'].apply(cleanText)
df_news.columns = ['Title','Media', 'Date', 'Desc']
df_news['Date'] = df_news['Date'].apply(cleanTime)

dataSetCombinadoGoogle = df_google.merge(df_market, how='inner',on='Date')
dataSetCombinadoTwitter = df_tweet.merge(df_market, how='inner', on='Date')

dataSetCombinadoTwitter = labeleador(dataSetCombinadoTwitter)
dataSetCombinadoGoogle = limpiaErroresG(dataSetCombinadoGoogle)
dataSetCombinadoGoogle = labeleador(dataSetCombinadoGoogle)





No keys, only access to public API functions


In [183]:
bow_vectorizerT = CountVectorizer(max_df=0.9, min_df=2, max_features=1000, stop_words='english')
bowT = bow_vectorizerT.fit_transform(dataSetCombinadoTwitter['Tweets'])

test = train_test_split(bowT,dataSetCombinadoTwitter['Label'], random_state = 42, test_size=0.25)

bow_vectorizerG = CountVectorizer(max_df=0.9, min_df=2, max_features=1000, stop_words='english')
bowG = bow_vectorizerG.fit_transform(dataSetCombinadoGoogle['Desc'])

#test = train_test_split(bowG,dataSetCombinadoGoogle['Label'], random_state = 42, test_size=0.25)

In [184]:
predRegT = modelLRT.predict(test)

C:\Users\christian\miniconda3\lib\site-packages\sklearn\utils\validation.py:673: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  array = np.asarray(array, order=order, dtype=dtype)


ValueError: Expected 2D array, got 1D array instead:
array=[<375x765 sparse matrix of type '<class 'numpy.int64'>'
 	with 3366 stored elements in Compressed Sparse Row format>
 <125x765 sparse matrix of type '<class 'numpy.int64'>'
 	with 1125 stored elements in Compressed Sparse Row format>
 227    0
 417    0
 203    0
 126    0
 329    1
       ..
 106    0
 270    0
 348    1
 435    0
 102    1
 Name: Label, Length: 375, dtype: int64
 361    0
 73     1
 374    1
 155    0
 104    1
       ..
 220    0
 176    1
 320    0
 153    0
 231    1
 Name: Label, Length: 125, dtype: int64].
Reshape your data either using array.reshape(-1, 1) if your data has a single feature or array.reshape(1, -1) if it contains a single sample.